In [10]:
#import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from pprint import pprint

In [11]:
# load ascii text and covert to lowercase
filename = "FAHRENHEIT 451.txt"
raw_text = open(filename, 'rb').read()
#raw_text = raw_text.lower()
pprint(raw_text)

(b'FAHRENHEIT 451 by Ray Bradbury\nThis one, with gratitude, is for DON CONG'
 b'DON.\nFAHRENHEIT 451:\nThe temperature at which book-paper catches fire an'
 b'd burns\nPART I\nIT WAS A PLEASURE TO BURN\nIT was a special pleasure to se'
 b'e things eaten, to see things blackened and changed. With the brass nozzle i'
 b'n his fists, with this great python spitting its venomous kerosene upon the '
 b'world, the blood pounded in his head, and his hands were the hands of some a'
 b'mazing conductor playing all the symphonies of blazing and burning to bring '
 b'down the tatters and charcoal ruins of history. With his symbolic helmet num'
 b'bered 451 on his stolid head, and his eyes all orange flame with the thought'
 b' of what came next, he flicked the igniter and the house jumped up in a gorg'
 b'ing fire that burned the evening sky red and yellow and black. He strode in '
 b'a swarm of fireflies. He wanted above all, like the old joke, to shove a mar'
 b'shmallow on a stick in the furn

 b'itten a book on tobacco and cancer of the lungs? The cigarette people are we'
 b'eping? Bum the book. Serenity, Montag. Peace, Montag. Take your fight outsid'
 b'e. Better yet, into the incinerator. Funerals are unhappy and pagan? Elimina'
 b"te them, too. Five minutes after a person is dead he's on his way to the Big"
 b' Flue, the Incinerators serviced by helicopters all over the country. Ten mi'
 b"nutes after death a man's a speck of black dust. Let's not quibble over indi"
 b'viduals with memoriams. Forget them. Burn them all, burn everything. Fire is'
 b' bright and fire is clean."\nThe fireworks died in the parlour behind Mil'
 b'dred. She had stopped talking at the same time; a miraculous coincidence. Mo'
 b'ntag held his breath.\n"There was a girl next door," he said, slowly. "Sh'
 b"e's gone now, I think, dead. I can't even remember her face. But she was dif"
 b'ferent. How?how did she happen?"\nBeatty smiled. "Here or there, that\'s b'
 b"ound to occur. Clarisse McClellan

 b" at the mess, and what do you do? Pride, damn it, and temper, and you've jun"
 b'ked it all, at the very start you vomit on everyone and on yourself. But eve'
 b'rything\nat once, but everything one on top of another; Beatty, the women'
 b', Mildred, Clarisse, everything. No excuse, though, no excuse. A fool, a dam'
 b"n fool, go give yourself up!\nNo, we'll save what we can, we'll do what t"
 b"here is left to do. If we have to burn, let's take a few more with us. Here!"
 b'\nHe remembered the books and turned back. Just on the off chance.\nHe fou'
 b'nd a few books where he had left them, near the garden fence. Mildred, God b'
 b'less her, had missed a few. Four books still lay hidden where he had put the'
 b'm. Voices were wailing in the\nnight and flashbeams swirled about. Other '
 b'Salamanders were roaring their engines far away, and police sirens were cutt'
 b'ing their way across town with their sirens.\nMontag took the four remain'
 b'ing books and hopped, jolted, hopped hi

In [12]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

{10: 0, 32: 1, 33: 2, 34: 3, 39: 4, 40: 5, 41: 6, 44: 7, 45: 8, 46: 9, 48: 10, 49: 11, 50: 12, 51: 13, 52: 14, 53: 15, 54: 16, 55: 17, 56: 18, 57: 19, 58: 20, 59: 21, 63: 22, 65: 23, 66: 24, 67: 25, 68: 26, 69: 27, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 77: 35, 78: 36, 79: 37, 80: 38, 81: 39, 82: 40, 83: 41, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 96: 48, 97: 49, 98: 50, 99: 51, 100: 52, 101: 53, 102: 54, 103: 55, 104: 56, 105: 57, 106: 58, 107: 59, 108: 60, 109: 61, 110: 62, 111: 63, 112: 64, 113: 65, 114: 66, 115: 67, 116: 68, 117: 69, 118: 70, 119: 71, 120: 72, 121: 73, 122: 74}


In [13]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  251222
Total Vocab:  75


In [14]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 256
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  250966


In [15]:
print(dataX[1])

[23, 30, 40, 27, 36, 30, 27, 31, 42, 1, 14, 15, 11, 1, 50, 73, 1, 40, 49, 73, 1, 24, 66, 49, 52, 50, 69, 66, 73, 0, 42, 56, 57, 67, 1, 63, 62, 53, 7, 1, 71, 57, 68, 56, 1, 55, 66, 49, 68, 57, 68, 69, 52, 53, 7, 1, 57, 67, 1, 54, 63, 66, 1, 26, 37, 36, 1, 25, 37, 36, 29, 26, 37, 36, 9, 0, 28, 23, 30, 40, 27, 36, 30, 27, 31, 42, 1, 14, 15, 11, 20, 0, 42, 56, 53, 1, 68, 53, 61, 64, 53, 66, 49, 68, 69, 66, 53, 1, 49, 68, 1, 71, 56, 57, 51, 56, 1, 50, 63, 63, 59, 8, 64, 49, 64, 53, 66, 1, 51, 49, 68, 51, 56, 53, 67, 1, 54, 57, 66, 53, 1, 49, 62, 52, 1, 50, 69, 66, 62, 67, 0, 38, 23, 40, 42, 1, 31, 0, 31, 42, 1, 45, 23, 41, 1, 23, 1, 38, 34, 27, 23, 41, 43, 40, 27, 1, 42, 37, 1, 24, 43, 40, 36, 0, 31, 42, 1, 71, 49, 67, 1, 49, 1, 67, 64, 53, 51, 57, 49, 60, 1, 64, 60, 53, 49, 67, 69, 66, 53, 1, 68, 63, 1, 67, 53, 53, 1, 68, 56, 57, 62, 55, 67, 1, 53, 49, 68, 53, 62, 7, 1, 68, 63, 1, 67, 53, 53, 1, 68, 56, 57, 62, 55, 67, 1, 50, 60, 49, 51, 59, 53, 62, 53, 52, 1, 49]


In [16]:
print(dataY[1])

62


In [17]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(128,input_shape=(X.shape[1], X.shape[2]), return_sequences=True)) 
#model.add(LSTM(512,input_shape=(X.shape[1], X.shape[2]))) 
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-128->512.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=8, callbacks=callbacks_list)